In [1]:
#Retrieve and save data locally and if possible, upload to AWS S3.

This is a comprehensive overview of how to pre-treat medical lung images prior to them being fed into D.L. machine.

This is the outline if what will be accomplished:

1. **Loading the DICOM files** and adding missing metadata.
2. **Converting the pixel values to Hounsfield Units (HU),** and to what tissue these unit values correspond.
3. **Resampling** to an isomorphic resolution to remove variance in scanner resolution.
4. **3D plotting.** (Visualization is very useful to see what we are doing).
5. **Lung segmentation.**
6. **Normalization** that makes sense.
7. **Zero centering** the scans.

Importing pkgs and determining Pat's.

In [6]:
%matplotlib inline

import numpy as np
import pandas as pd
import dicom
import os
import scipy.ndimage
import matplotlib.pyplot as plt

from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

#Some constants
# INPUT_FOLDER = '../input/sample_images/'
# patients = os.listdir(INPUT_FOLDER)
# patients.sort()

## Loading the Files

Dicom 
- de-facto file standard for medical imaging. 
- contain a lot of metadata (pixel size, how long one pixel is in every dimension in the real world)
- pixel size/coarseness of scan differs from scan to scan (perform isomorphic resampling)

Following is code to load scan, which has multiple slices. 
This will be saved in a Python list.
Every folder in dataset is one scan (one patient).
One metadata field is missing, pixel size in Z direction, which is the slice thickness.
This will be inferred and added to metadata. 

In [10]:
#Load scans in given folder path
def load_scan(path):
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0]).SliceLocation - slices[1].SliceLocation
    
    for s in slices:
        s.SliceThickness = slice_thickness
                                        
    return slices

The unit of measurement in CT scans is **Hounsfield Unit (HU)** which is the measurement of radiodensity. CT scanners are carefully calibrated to accurately measure this. 

By default though, the returned values are not in this unit. This can be fixed.

Some scanners have cylindrical scanning bounds, but the output image is square. The pixels that fall outside of these bounds get the fixed value -2000. This will be set to 0, which corresponds to air. 

Getting back to HU units, we will multiply with rescale slope and adding the intercept (which is stored in metadata of the scans).

In [18]:
def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices])
    #Convert to int16 (from sometimes int16),
    #should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)
    
    #Set outside-of-scan pixels to 0
    #The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    #Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):
        
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
            
        image[slice_number] += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

Taking a look at one of the patients:

In [20]:
first_patient = load_scan(INPUT_FOLDER + patients[0])
first_patient_pixels = get_pixels_hu(first_patient)
plt.hist(first_patient_pixels.flatten(), bins = 80, color = 'c')
plt.xlabel("Hounsfield Units (HU)")
plt.ylabel("Frequency")
plt.show()

# #Show some slice in the middle
plt.imshow(first_patient_pixels[80], cmp=plt.cm.gray)
plt.show()

In [23]:
first_patient = load_scan(INPUT_FOLDER + patients[0])
first_patient_pixels = get_pixels_hu(first_patient)
plt.hist(first_patient_pixels.flatten(), bins = 80, color = 'c')
plt.xlabel("Hounsfield Units (HU)")
plt.ylabel("Frequency")
plt.show()

# #Show slice in the middle
plt.imshow(first_patient_pixels[80], cmap=plt.cm.gray)
plt.show()

Based on Hounsfield Units and this histogram, clearly see which pixels are air and which are tissue.